In [1]:
state = {
    "initial_ad": str,   # original ad
    "tone": str,         # e.g. "fun"
    "platform": str,     # e.g. "Instagram"
    "polished_ad": str,  # latest polished ad
    "is_ok": bool,       # review outcome
    "feed_back": str     # feedback for improvement
}


In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os
llm=ChatGroq(model="llama3-8b-8192",api_key=os.getenv("GROQ_API_KEY"))


In [21]:
from pydantic import BaseModel , Field
from typing import Literal

class State(BaseModel):
    initial_ad: str = Field(description="The original ad")
    tone: str = Field(description="The tone of the ad")
    platform: str = Field(description="The platform for the ad")
    polished_ad: str = Field(description="The polished ad")
    is_ok: bool = Field(description="Whether the ad is ok")
    feed_back: str = Field(description="The feedback for the ad")


In [12]:
add_writer_prompt="""
You are a professional ad writer.
You are given an ad and you need to write a new ad for the same product.
The ad should be in the same tone as the original ad.
The ad should be in the same platform as the original ad.
The ad should be in the same tone as the original ad.
You should be very creative and unique in your writing.


Here is the original ad:
{initial_ad}

You might also have some feedback for the ad.
{feedback}
"""

In [29]:
review_prompt="""
You are a professional ad reviewer.
You are given an ad and you need to review it.
You need to check if the ad is well-written and suitable for the platform and tone.
If you it is good then return True else return False. remember only return True or False.

Here is the ad:
{polished_ad}

The tone of the ad is {tone} and the platform is {platform}.
"""

In [16]:
feedback_prompt="""
You are a professional ad reviewer.
You are given an ad and you need to review it.
You need to check if the ad is well-written and suitable for the platform and tone.
You need to provide actionable feedback to improve the ad.
You need to provide feedback in the same tone as the original ad.
You need to provide feedback in the same platform as the original ad.

Here is the ad:
{polished_ad}

"""

In [22]:
class reviw_feedback(BaseModel):
  review:bool=Field(description="Whether the ad is good or not")

In [23]:
llm_new=llm.with_structured_output(reviw_feedback)


In [24]:
polished="often the best way to learn is by doing"

In [26]:
prompt=review_prompt.format(polished_ad=polished)
llm_new=llm.with_structured_output(reviw_feedback)
is_ok = llm_new.invoke(prompt)


In [28]:
is_ok.review

True

In [33]:
def polish_node(state):
    text = state["polished_ad"] if state.get("polished_ad") else state["initial_ad"]
    feedback = state.get("feed_back", "")
    prompt=add_writer_prompt.format(initial_ad=text,feedback=feedback)
    polished = llm.invoke(prompt)
    state["polished_ad"] = polished.content
    return state

def review_node(state):
    polished = state["polished_ad"]
    prompt=review_prompt.format(polished_ad=polished,tone=state["tone"],platform=state["platform"])
    llm_new=llm.with_structured_output(reviw_feedback)
    is_ok = llm_new.invoke(prompt).review
    state["is_ok"] = is_ok
    return state

def feedback_node(state):
    polished = state["polished_ad"]
    prompt=feedback_prompt.format(polished_ad=polished)
    feedback = llm.invoke(prompt).content
    state["feed_back"] = feedback
    return state


In [34]:
from langgraph.graph import StateGraph, END

def build_graph():
    graph = StateGraph(dict)
    graph.add_node("polish", polish_node)
    graph.add_node("review", review_node)
    graph.add_node("feedback", feedback_node)

    # Edges
    graph.add_edge("polish", "review")
    graph.add_conditional_edges(
        "review",
        lambda state: "end" if state["is_ok"] else "feedback",
        {"end": END, "feedback": "feedback"}
    )
    graph.add_edge("feedback", "polish")

    graph.set_entry_point("polish")
    return graph.compile()


In [35]:
state = {
    "initial_ad": "Big Sale! Best prices on shoes. Visit now!",
    "tone": "fun",
    "platform": "Instagram",
    "polished_ad": "",
    "is_ok": False,
    "feed_back": "",
}
graph = build_graph()
final_state = graph.invoke(state)
print("Final polished ad:", final_state["polished_ad"])

Final polished ad: Thank you for the opportunity to rewrite the ad. Based on the original ad, I understand that the tone is casual and attention-grabbing, and the platform is likely social media or a website. Here's a rewritten ad that maintains the same tone and platform:

KICK IT UP A NOTCH!

Sole Searching? Look no further!

We're throwing a shoe-tastic sale! Get ready to step up your shoe game with our unbeatable prices! From sleek sneakers to stylish stilettos, our collection has got you covered.

Limited time offer! Don't miss out on this chance to elevate your shoe wardrobe without breaking the bank. Visit us now and get ready to kick it up a notch!

Feedback on the original ad:

* The ad is short and to the point, but it lacks creativity and attention-grabbing elements.
* It's a good idea to highlight the "best prices" but it would be more effective to specify what makes the prices unbeatable.
* Adding a sense of urgency with a limited-time offer can encourage customers to take